## 0. Importacion y Ejecucion del env
- Se importan componentes de carga, segmentación, embeddings (Ollama), vectorstore (Chroma) y LLM (Gemini) para armar el flujo RAG.

In [32]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, PDFPlumberLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import os

from dotenv import load_dotenv
load_dotenv()

api_key =os.getenv("API_KEY")

## 1. Implementación Práctica de RAG:


In [33]:
def cargar_pdf(ruta_pdf: str):
    loader = PyPDFLoader(ruta_pdf)
    docs = loader.load()  # documentos por página con metadata
    return docs

text = cargar_pdf("seguridad.pdf")

def partir_texto(docs_or_text):
    splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
    
    # aceptar tanto lista de Documents como texto plano
    if isinstance(docs_or_text, list):
        texts = [d.page_content for d in docs_or_text]
    else:
        texts = [docs_or_text]
        
    docs = splitter.create_documents(texts)
    return docs

docs = partir_texto(text)

def crear_embeddings():
    embedding = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)


embedding = crear_embeddings()

def crear_vectorstore(docs, embedding):
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embedding,
        collection_name="base_rag_mem_384"
    )
    return vectorstore

vectorstore = crear_vectorstore(docs, embedding)

def recuperar_contexto(pregunta, vectorstore):
    docs_rel = vectorstore.similarity_search(pregunta, k=3)
    contexto = "\n".join([d.page_content for d in docs_rel])
    return contexto

pregunta = "¿Que es el Command Injection?"
contexto = recuperar_contexto(pregunta, vectorstore)

def responder_llm(pregunta, contexto):
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.3,
        api_key=os.getenv("API_KEY")
    )
    prompt = (
        f"Eres un asistente sobre LangChain.\n"
        f"Responde usando sólo este contexto:\n{contexto}\n"
        f"Pregunta: {pregunta}"
    )
    respuesta = llm.invoke(prompt)
    return respuesta.content

respuesta = responder_llm(pregunta, contexto)
print(respuesta)


Command Injection (OS Command Injection) es una inyección que permite ejecutar comandos del sistema operativo en el servidor.


## 2. Exploración de Embeddings y LLMs

In [29]:
#Varios embeddings
embedding_models = ["nomic-embed-text", "mxbai-embed-large", "all-minilm"]

def probar_embeddings(pregunta):
    resultados = {}
    for model in embedding_models:
        emb = OllamaEmbeddings(model=model)
        vs = Chroma.from_documents(docs, emb, collection_name=f"prueba_{model}")
        docs_rel = vs.similarity_search(pregunta, k=3)
        resultados[model] = [d.page_content for d in docs_rel]
    return resultados

res_emb = probar_embeddings("¿que es Injection (familia)?")
print(res_emb)


{'nomic-embed-text': ['Técnico Superior en Desarrollo de Software Multiplataforma    \nSeminario Actualización III – Ciberseguridad \nProfesores: Alejandro Sanabria – Alejandro Ruíz Díaz \n1.A SQL Injection (SQLi) \n• Ejemplo (PHP vulnerable): \n$q = "SELECT * FROM users WHERE username = \'" . $_GET[\'user\'] . "\' AND \npassword = \'" . $_GET[\'pass\'] . "\'"; \n• Payload — login bypass: user=admin\' -- &pass=x \n• Payload — extracción (union): id=1 UNION SELECT user, password FROM \nusers-- \n• Detección: insertar \' o OR 1=1 y observar errores o comportamientos distintos. \nPruebas blind (time-based) con SLEEP(5). \n• Mitigación: prepared statements / parametrized queries, validación estricta, \nmenor privilegio para la cuenta DB. \n1.B NoSQL Injection (MongoDB ejemplo) \n• Ejemplo (Node.js vulnerable): \ndb.users.find({ username: req.body.username, password: req.body.password \n}) \n• Payload (JSON): {"username":{"$ne":null},"password":{"$ne":null}} → \ndevuelve documentos sin comp

In [30]:
llm_models = {
    "gemini-2.5-flash": ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2, api_key=os.getenv("API_KEY")),
    "gemini-2.5-pro": ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.2, api_key=os.getenv("API_KEY")),
}

def probar_llms(pregunta, contexto):
    resultados = {}
    for nombre, llm in llm_models.items():
        prompt = f"Contexto:\n{contexto}\nPregunta: {pregunta}"
        respuesta = llm.invoke(prompt)
        resultados[nombre] = respuesta.content
    return resultados

ctx = recuperar_contexto("Dime que es Cross-Site Scripting", vectorstore)
res_llm = probar_llms("Dime que es Cross-Site Scripting", ctx)
print(res_llm)


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 6.333676946s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 2
}
, retry_delay {
  seconds:

KeyboardInterrupt: 

### 3. Optimizacion del separador de texto

In [36]:
def probar_splitter_sizes(text):
    tamaños = [(80, 10), (150, 20), (300, 40)]
    resultados = {}

    # Normalizar la entrada a una lista de strings que CharacterTextSplitter espera
    if isinstance(text, list):
        # Si es una lista de Document objetos, extraer page_content
        if len(text) > 0 and hasattr(text[0], "page_content"):
            texts = [d.page_content for d in text]
        else:
            texts = [str(t) for t in text]
    else:
        texts = [text]

    for size, overlap in tamaños:
        splitter = CharacterTextSplitter(chunk_size=size, chunk_overlap=overlap)
        docs = splitter.create_documents(texts)
        n_docs = len(docs) if docs else 0
        avg_len = sum(len(d.page_content) for d in docs)/n_docs if n_docs else 0
        resultados[(size, overlap)] = {"n_chunks": n_docs, "avg_len": avg_len}
    return resultados

split_stats = probar_splitter_sizes(text)
print(split_stats)


{(80, 10): {'n_chunks': 7, 'avg_len': 1605.2857142857142}, (150, 20): {'n_chunks': 7, 'avg_len': 1605.2857142857142}, (300, 40): {'n_chunks': 7, 'avg_len': 1605.2857142857142}}


## 4. Gestion de bases vectoriales


In [38]:
def chunks_con_metadatos(docs):
    for i, d in enumerate(docs):
        d.metadata = {"chunk_id": i, "tema": "Command Injection"}
    return docs

docs_meta = chunks_con_metadatos(docs)

vect_meta = Chroma.from_documents(
    documents=docs_meta,
    embedding=embedding,
    collection_name="meta_rag"
)

filtrados = vect_meta.similarity_search("¿Command Injection?", k=3, filter={"tema": "Command Injection"})
for d in filtrados:
    print(d.metadata, d.page_content)


{'tema': 'Command Injection', 'chunk_id': 2} Técnico Superior en Desarrollo de Software Multiplataforma    
Seminario Actualización III – Ciberseguridad 
Profesores: Alejandro Sanabria – Alejandro Ruíz Díaz 
2. Command Injection (OS Command Injection) 
Definición: Inyección que permite ejecutar comandos del sistema operativo en el 
servidor. 
Cómo funciona 
• Ocurre cuando la aplicación pasa input directamente a un intérprete de 
comandos (shell) sin separar o sanear. 
Ejemplo 
• Código PHP vulnerable: 
$output = shell_exec('ping -c 1 ' . $_GET['host']); 
• Payload: 8.8.8.8; cat /etc/passwd 
• Resultado: el servidor ejecuta ping -c 1 8.8.8.8; cat /etc/passwd y 
devuelve el contenido de /etc/passwd. 
Detección 
• Intentar introducir ;, &&, |, backticks o caracteres especiales y observar si 
aparecen resultados de comandos en la salida. 
Mitigación 
• Evitar invocar shell; usar APIs nativas que no ejecuten un intérprete. Si es 
imprescindible, whitelist de hosts/inputs, escapes robustos 

## 5. Refinamiento de prompts


In [40]:
# Prompt con bullets
prompt_bullets = ChatPromptTemplate.from_messages([
    ("system", "Responde con bullets y cita fragmentos. Si no está, di 'No está en el documento'."),
    ("human", "Pregunta: {question}\nContexto:{context}\nFormato: - punto 1 [Fuente]")
])

def responder_bullets(pregunta, contexto):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, api_key=os.getenv("API_KEY"))
    formatted_prompt = f"{pregunta}\nContexto:{contexto}"
    return llm.invoke(formatted_prompt).content

print(responder_bullets("Explicame que es Insecure Direct Object Reference (IDOR)", contexto))


¡Excelente! Basándonos en el formato y nivel de detalle de los ejemplos que has proporcionado (Command Injection y XSS), aquí tienes la explicación de Insecure Direct Object Reference (IDOR):

---

**4. Insecure Direct Object Reference (IDOR)**

**Definición:**
Una vulnerabilidad que ocurre cuando una aplicación web expone una referencia directa a un objeto interno (como un archivo, una entrada de base de datos, una cuenta de usuario, un pedido, etc.) y no implementa controles de autorización adecuados para verificar si el usuario que realiza la solicitud tiene permiso para acceder a ese objeto específico. Esto permite a un atacante manipular estas referencias para acceder a recursos a los que no debería tener acceso.

**Cómo funciona:**
*   La aplicación utiliza un identificador directo (como un ID numérico, un nombre de archivo o un UUID) para acceder a un recurso.
*   Este identificador se expone a menudo en la URL, en parámetros de formularios (GET o POST), en el cuerpo de una soli

In [41]:
# Prompt con verificación
prompt_verif = ChatPromptTemplate.from_messages([
    ("system", "Responde breve y agrega al final 'Verificado con contexto'."),
    ("human", "Pregunta: {question}\nContexto:{context}")
])

def responder_verif(pregunta, contexto):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.1, api_key=os.getenv("API_KEY"))
    formatted_prompt = f"{pregunta}\nContexto:{contexto}"
    return llm.invoke(formatted_prompt).content

print(responder_verif("Dime ejemplos sobre Insecure Direct Object Reference (IDOR)", contexto))


¡Claro! Siguiendo el formato y el contexto de tu seminario, aquí tienes una sección completa sobre **Insecure Direct Object Reference (IDOR)**, lista para ser añadida a tus apuntes.

---

### **4. Insecure Direct Object Reference (IDOR)**

**Definición:**
Es una vulnerabilidad de control de acceso que ocurre cuando una aplicación utiliza un identificador de un objeto (como un ID de usuario, número de factura o nombre de archivo) proporcionado por el usuario para acceder directamente a un recurso, sin verificar si dicho usuario tiene los permisos necesarios para hacerlo.

**Cómo funciona:**
1.  La aplicación expone una referencia directa a un objeto interno en una URL o parámetro (ej: `?user_id=123`).
2.  Un atacante modifica el valor de esta referencia para apuntar a un objeto que no le pertenece (ej: cambia `?user_id=123` por `?user_id=456`).
3.  El servidor, al no realizar una comprobación de autorización adecuada, procesa la solicitud y devuelve los datos del objeto solicitado (los 

In [42]:
# Prompt con fuentes
prompt_fuentes = ChatPromptTemplate.from_messages([
    ("system", "Al final incluye 'Fuentes' con los fragmentos usados."),
    ("human", "Pregunta: {question}\nContexto:{context}")
])

def responder_fuentes(pregunta, contexto):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2, api_key=os.getenv("API_KEY"))
    formatted_prompt = f"{pregunta}\nContexto:{contexto}"
    return llm.invoke(formatted_prompt).content

print(responder_fuentes("Resume el documento en 2 frases.", contexto))


El documento detalla dos vulnerabilidades de ciberseguridad: Command Injection, que permite ejecutar comandos del sistema operativo en el servidor por falta de saneamiento de entradas, y Cross-Site Scripting (XSS), que inyecta código malicioso en el navegador de otros usuarios. Ambas se detectan mediante el envío de payloads de prueba y se mitigan evitando la ejecución directa de entradas no confiables y saneando rigurosamente los datos.
